In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Load and preprocess the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the VGG-style model architecture
def VGG():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Compile the model
model = VGG()
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=20, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the results
results = {
    'test_loss': test_loss,
    'test_accuracy': test_acc,
    'history': history.history
}

import json
with open('vgg_fashion_mnist_results.json', 'w') as f:
    json.dump(results, f)


26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


c:\Python\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 73s 189ms/step - accuracy: 0.6325 - loss: 0.9727 - val_accuracy: 0.8547 - val_loss: 0.3773
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 79s 210ms/step - accuracy: 0.8578 - loss: 0.3842 - val_accuracy: 0.8872 - val_loss: 0.3009
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 73s 186ms/step - accuracy: 0.8834 - loss: 0.3165 - val_accuracy: 0.9034 - val_loss: 0.2595
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 188ms/step - accuracy: 0.8975 - loss: 0.2745 - val_accuracy: 0.9100 - val_loss: 0.2398
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 189ms/step - accuracy: 0.9060 - loss: 0.2541 - val_accuracy: 0.9180 - val_loss: 0.2287
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 184ms/step - accuracy: 0.9132 - loss: 0.2349 - val_accuracy: 0.9198 - val_loss: 0.2277
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 184ms/step - accuracy: 0.9190 - loss: 0.2201 - val_accuracy: 0.9253 - val_loss: 0.2028
Epoch 8/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 184ms/step - accuracy: 0.9207 - loss: 0

# REsnet with FMNSIT

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Load and preprocess the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the ResNet model architecture
def resnet_block(input_tensor, filters, kernel_size=3, stride=1):
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    if stride != 1:
        input_tensor = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(input_tensor)
        input_tensor = BatchNormalization()(input_tensor)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

def ResNet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, kernel_size=3, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = resnet_block(x, 32)
    x = resnet_block(x, 32)
    x = resnet_block(x, 64, stride=2)
    x = resnet_block(x, 64)
    x = resnet_block(x, 128, stride=2)
    x = resnet_block(x, 128)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, x)
    return model

# Compile the model
model = ResNet(input_shape=(28, 28, 1), num_classes=10)
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=20, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the results
results = {
    'test_loss': test_loss,
    'test_accuracy': test_acc,
    'history': history.history
}

import json
with open('resnet_fashion_mnist_results.json', 'w') as f:
    json.dump(results, f)


Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 401s 1s/step - accuracy: 0.7075 - loss: 1.5427 - val_accuracy: 0.4660 - val_loss: 1.4488
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 577s 1s/step - accuracy: 0.8974 - loss: 0.2785 - val_accuracy: 0.8453 - val_loss: 0.4260
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 275s 615ms/step - accuracy: 0.9198 - loss: 0.2206 - val_accuracy: 0.9079 - val_loss: 0.2532
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 234s 625ms/step - accuracy: 0.9385 - loss: 0.1715 - val_accuracy: 0.9028 - val_loss: 0.3036
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 234s 625ms/step - accuracy: 0.9479 - loss: 0.1432 - val_accuracy: 0.9168 - val_loss: 0.2427
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 236s 630ms/step - accuracy: 0.9542 - loss: 0.1223 - val_accuracy: 0.9193 - val_loss: 0.2401
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 237s 633ms/step - accuracy: 0.9647 - loss: 0.1003 - val_accuracy: 0.9193 - val_loss: 0.2508
Epoch 8/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 243s 647ms/step - accuracy: 0.9674 - loss:

## LENET WITH FMNSIT

In [4]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Load and preprocess the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the LeNet model architecture
def LeNet():
    model = Sequential([
        Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
        AveragePooling2D(pool_size=(2, 2)),
        Conv2D(16, kernel_size=(5, 5), activation='relu'),
        AveragePooling2D(pool_size=(2, 2), ),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(10, activation='softmax')
    ])
    return model

# Compile the model
model = LeNet()
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the results
results = {
    'test_loss': test_loss,
    'test_accuracy': test_acc,
    'history': history.history
}

import json
with open('lenet_fashion_mnist_results.json', 'w') as f:
    json.dump(results, f)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6201 - loss: 1.0662 - val_accuracy: 0.7902 - val_loss: 0.5754
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7995 - loss: 0.5379 - val_accuracy: 0.8277 - val_loss: 0.4775
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8328 - loss: 0.4569 - val_accuracy: 0.8358 - val_loss: 0.4407
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8501 - loss: 0.4170 - val_accuracy: 0.8506 - val_loss: 0.4134
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8622 - loss: 0.3821 - val_accuracy: 0.8536 - val_loss: 0.4096
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8655 - loss: 0.3640 - val_accuracy: 0.8633 - val_loss: 0.3790
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8727 - loss: 0.3449 - val_accuracy: 0.8637 - val_loss: 0.3649
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8786 - loss: 0.3317 - val_accuracy: 0.

## Googlenet WITH FMNSIT

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Load and preprocess the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Define the Inception module
def inception_module(x, filters):
    branch1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)
    
    branch3x3 = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    branch3x3 = Conv2D(filters[1], (3, 3), padding='same', activation='relu')(branch3x3)
    
    branch5x5 = Conv2D(filters[2], (1, 1), padding='same', activation='relu')(x)
    branch5x5 = Conv2D(filters[2], (5, 5), padding='same', activation='relu')(branch5x5)
    
    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(branch_pool)
    
    # Concatenate all the branches along the depth axis
    return concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)

# Define the GoogLeNet model
def googlenet(input_shape=(28, 28, 1), num_classes=10):
    inputs = Input(shape=input_shape)

    # Initial Convolution Layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = AveragePooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # First Inception module
    x = inception_module(x, filters=(32, 64, 16, 16))
    
    # Second Inception module
    x = inception_module(x, filters=(64, 128, 32, 32))

    # Global Average Pooling
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Fully connected layers
    x = Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    # Output layer
    output = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs, output)

    return model

# Instantiate the GoogLeNet model
model = googlenet(input_shape=(28, 28, 1), num_classes=10)

# Compile the model
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the results
results = {
    'test_loss': test_loss,
    'test_accuracy': test_acc,
    'history': history.history
}

import json
with open('googlenet_fashion_mnist_results.json', 'w') as f:
    json.dump(results, f)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 35s 82ms/step - accuracy: 0.3566 - loss: 1.6248 - val_accuracy: 0.7027 - val_loss: 0.7477
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 28s 75ms/step - accuracy: 0.6992 - loss: 0.7850 - val_accuracy: 0.7717 - val_loss: 0.5970
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 28s 75ms/step - accuracy: 0.7549 - loss: 0.6461 - val_accuracy: 0.8039 - val_loss: 0.5133
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 28s 76ms/step - accuracy: 0.7868 - loss: 0.5808 - val_accuracy: 0.8065 - val_loss: 0.4972
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 29s 79ms/step - accuracy: 0.8136 - loss: 0.5195 - val_accuracy: 0.8372 - val_loss: 0.4319
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 29s 77ms/step - accuracy: 0.8296 - loss: 0.4816 - val_accuracy: 0.8499 - val_loss: 0.3990
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 29s 77ms/step - accuracy: 0.8420 - loss: 0.4467 - val_accuracy: 0.8540 - val_loss: 0.3953
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 29s 77ms/step - accuracy: 0.8545 - loss: 0.4072 - 

## Xceptionnet withFMNSIT

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense, Input, SeparableConv2D, MaxPooling2D, BatchNormalization, Activation, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Load and preprocess the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Convert grayscale images to RGB (3 channels)
train_images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(train_images))
test_images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(test_images))

# Resize images to the input size required by Xception (71x71)
train_images = tf.image.resize(train_images, [71, 71])
test_images = tf.image.resize(test_images, [71, 71])


#Define xception block
def xception_block(x, filters, strides=(1, 1), shortcut_connection=True):
    residual = x

    # Separable Conv 1
    x = SeparableConv2D(filters, (3, 3), padding='same', strides=strides)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Separable Conv 2
    x = SeparableConv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    # Shortcut connection
    if shortcut_connection:
        # Adjust residual path to match dimensions if necessary
        if strides != (1, 1) or residual.shape[-1] != filters:
            residual = Conv2D(filters, (1, 1), strides=strides, padding='same')(residual)
            residual = BatchNormalization()(residual)
        x = Add()([x, residual])  # Add the residual connection
    x = Activation('relu')(x)

    return x


# Define the Xception CNN model
def xception_cnn(input_shape=(71, 71, 3), num_classes=10):
    inputs = Input(shape=input_shape)

    # Initial Convolution Layer
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # MaxPooling Layer
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Xception Blocks
    x = xception_block(x, filters=64, shortcut_connection=False)
    x = xception_block(x, filters=128, strides=(2, 2))
    x = xception_block(x, filters=256, strides=(2, 2))
    x = xception_block(x, filters=728, strides=(2, 2))

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Fully connected layers
    x = Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    # Output layer
    output = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs, output)

    return model


# Instantiate the Xception CNN model
model = xception_cnn(input_shape=(71, 71, 3), num_classes=10)

# Compile the model
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the results
results = {
    'test_loss': test_loss,
    'test_accuracy': test_acc,
    'history': history.history
}

import json
with open('xception_fashion_mnist_results.json', 'w') as f:
    json.dump(results, f)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 160s 395ms/step - accuracy: 0.6456 - loss: 1.0140 - val_accuracy: 0.1378 - val_loss: 2.7999
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 150s 400ms/step - accuracy: 0.8672 - loss: 0.4010 - val_accuracy: 0.8884 - val_loss: 0.3121
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 182s 345ms/step - accuracy: 0.8948 - loss: 0.3266 - val_accuracy: 0.8934 - val_loss: 0.3064
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 138s 334ms/step - accuracy: 0.9056 - loss: 0.2826 - val_accuracy: 0.8897 - val_loss: 0.3484
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 122s 326ms/step - accuracy: 0.9118 - loss: 0.2633 - val_accuracy: 0.8842 - val_loss: 0.4411
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 127s 339ms/step - accuracy: 0.9209 - loss: 0.2358 - val_accuracy: 0.8972 - val_loss: 0.2824
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 125s 334ms/step - accuracy: 0.9283 - loss: 0.2103 - val_accuracy: 0.8962 - val_loss: 0.2811
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 139s 326ms/step - accuracy: 0.9311 -

senet 

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense, Input, SeparableConv2D, MaxPooling2D, BatchNormalization, Activation, Add, Reshape, Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Load and preprocess the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Reshape and normalize the images
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Convert grayscale images to RGB (3 channels)
train_images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(train_images))
test_images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(test_images))

# Resize images to a larger size for better model flexibility
train_images = tf.image.resize(train_images, [71, 71])
test_images = tf.image.resize(test_images, [71, 71])

# Define the Squeeze-and-Excitation block
def se_block(input_tensor, reduction_ratio=16):
    filters = input_tensor.shape[-1]  # Number of channels
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // reduction_ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape([1, 1, filters])(se)
    return Multiply()([input_tensor, se])

# Define a basic convolutional block with an SE block
def senet_block(x, filters, strides=(1, 1), shortcut_connection=True):
    residual = x

    # Convolutional layers
    x = Conv2D(filters, (3, 3), padding='same', strides=strides)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    # Squeeze-and-Excitation
    x = se_block(x)

    # Shortcut connection
    if shortcut_connection:
        if strides != (1, 1) or residual.shape[-1] != filters:
            residual = Conv2D(filters, (1, 1), strides=strides, padding='same')(residual)
            residual = BatchNormalization()(residual)
        x = Add()([x, residual])
    x = Activation('relu')(x)

    return x

# Define the SENet CNN model
def senet_cnn(input_shape=(71, 71, 3), num_classes=10):
    inputs = Input(shape=input_shape)

    # Initial Convolution Layer
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # MaxPooling Layer
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # SENet Blocks
    x = senet_block(x, filters=64, shortcut_connection=False)
    x = senet_block(x, filters=128, strides=(2, 2))
    x = senet_block(x, filters=256, strides=(2, 2))
    x = senet_block(x, filters=512, strides=(2, 2))

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Fully connected layers
    x = Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)

    # Output layer
    output = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs, output)

    return model

# Instantiate the SENet CNN model
model = senet_cnn(input_shape=(71, 71, 3), num_classes=10)

# Compile the model
model.compile(optimizer=Adam(),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

# Save the results
results = {
    'test_loss': test_loss,
    'test_accuracy': test_acc,
    'history': history.history
}

import json
with open('senet_fashion_mnist_results.json', 'w') as f:
    json.dump(results, f)


Epoch 1/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 394s 950ms/step - accuracy: 0.6887 - loss: 0.9096 - val_accuracy: 0.4277 - val_loss: 1.8577
Epoch 2/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 316s 841ms/step - accuracy: 0.8810 - loss: 0.3670 - val_accuracy: 0.8714 - val_loss: 0.3581
Epoch 3/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 308s 821ms/step - accuracy: 0.9012 - loss: 0.3011 - val_accuracy: 0.8757 - val_loss: 0.3598
Epoch 4/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 308s 822ms/step - accuracy: 0.9095 - loss: 0.2733 - val_accuracy: 0.9131 - val_loss: 0.2479
Epoch 5/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 299s 797ms/step - accuracy: 0.9182 - loss: 0.2396 - val_accuracy: 0.8819 - val_loss: 0.3207
Epoch 6/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 304s 810ms/step - accuracy: 0.9243 - loss: 0.2181 - val_accuracy: 0.8727 - val_loss: 0.3530
Epoch 7/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 302s 806ms/step - accuracy: 0.9326 - loss: 0.2007 - val_accuracy: 0.9095 - val_loss: 0.2680
Epoch 8/10
375/375 ━━━━━━━━━━━━━━━━━━━━ 302s 806ms/step - accuracy: 0.9373 -